# Preprocessing ABC notation files
- total 5267 rows

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
from collections import defaultdict
from typing import List, Dict

## 데이터 추출

In [2]:
def add_fields(fields:Dict[str,List[str]], lines:List[str]):
    if len(lines) > 0:
        info = defaultdict(lambda: None)
        for i, line in enumerate(lines):
            if line.startswith('T:'):
                info['T'] = line.split(':',1)[1].lstrip()
            if line.startswith('M:'):
                info['M'] = line.split(':',1)[1].lstrip()
            elif line.startswith('L:'):
                info['L'] = line.split(':',1)[1].lstrip()
            elif line.startswith('K:'):
                info['K'] = line.split(':',1)[1].lstrip()
                info['body'] = lines[i+1:]
                break
        [fields[key].append(info[key]) for key in fields.keys()]

    return fields, list()

In [3]:
fields = {'T':list(),'M':list(),'L':list(),'K':list(),'body':list()}
i = 0
for dir_info in os.walk('data'):
    for file_name in dir_info[2]:
        if file_name.startswith('.'):
            continue
        with open(os.path.join(dir_info[0],file_name),'r',encoding='ISO-8859-1') as f:
            lines = list()
            for line in f.readlines():
                if i > 100:
                    raise Exception()
                if line.startswith('%'):
                    continue
                elif line.strip() == str():
                    fields, lines = add_fields(fields, lines)
                else:
                    lines.append(line.strip())

### 데이터 확인

In [4]:
df = pd.DataFrame(fields)
print(df.shape)
df.head()

(7362, 5)


,T,M,L,K,body
0,The Enchanted Valley,2/4,1/16,Gm,[G3-A (Bcd=e) | f4 (g2dB) | ({d}c3-B) G2-E2 | ...
1,Fare You Well,2/4,1/16,D,[f-g | a3-b g3-a | f4 e3-d | d3-c A3-B | c4 d3...
2,The Little Heathy Hill,C,1/8,Gm,[B/2-c/2 | d2 d>-c B2 A-B | (GBAG) F2 D-F | (G...
3,The Little Girl of my Heart,4/4,1/8,D,[F-G | A-dd>-c d2 cd | e-fgg (f2 d>e) | f-d (c...
4,The Fun at Donnybrook,6/8,1/8,Gm,[B/2-A/2 | G>FD C>D^F | G3z2B/2-c/2 | B>cd cAG...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7362 entries, 0 to 7361
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   T       7362 non-null   object
 1   M       7362 non-null   object
 2   L       6676 non-null   object
 3   K       7362 non-null   object
 4   body    7362 non-null   object
dtypes: object(5)
memory usage: 287.7+ KB


### 중복된 데이터 제거

In [6]:
df = df[df['body'].duplicated().__invert__()]
print(df.shape)
df.head()

(5289, 5)


,T,M,L,K,body
0,The Enchanted Valley,2/4,1/16,Gm,[G3-A (Bcd=e) | f4 (g2dB) | ({d}c3-B) G2-E2 | ...
1,Fare You Well,2/4,1/16,D,[f-g | a3-b g3-a | f4 e3-d | d3-c A3-B | c4 d3...
2,The Little Heathy Hill,C,1/8,Gm,[B/2-c/2 | d2 d>-c B2 A-B | (GBAG) F2 D-F | (G...
3,The Little Girl of my Heart,4/4,1/8,D,[F-G | A-dd>-c d2 cd | e-fgg (f2 d>e) | f-d (c...
4,The Fun at Donnybrook,6/8,1/8,Gm,[B/2-A/2 | G>FD C>D^F | G3z2B/2-c/2 | B>cd cAG...


곡 제목을 학습에 활용할 경우 추가로 중복 제거

In [7]:
print(df[df['T'].duplicated()].shape)
df[df['T'].duplicated()].head()

(359, 5)


,T,M,L,K,body
7,The Little Fair Child,6/8,1/8,D,[A/A/ | d>ed cde | f>ed e2 f/-e/ | d>-cB/-G/ F...
12,The Red Haired Girl,3/4,1/8,D,[A/2-G/2 FA | B>-G E-D CE | D2 (E/2F/2G) F-A |...
24,The Black Slender Boy,3/4,1/8,G,"[G/2-A/2 Bc \, | d>-g (g/2f/2d/2c/2) Bc/2-A/2 ..."
25,The Black Slender Boy,3/4,1/8,G,"[c/2-A/2 (G/2A/2B/2c/2) \, | d>-g {ga}b-a g/2-..."
31,The Brown Thorn,3/4,1/8,D,[D-E | (F>E F)(ABc) | d3 c (BA/2F/2) | ~E4-D2 ...


우선, body 문자열만 가지고 Char RNN 시도

## 불용어 제거

In [8]:
char_vocab = {
    'C', 'D', 'E', 'F', 'G', 'A', 'B', 'c', 'd', 'e', 'f', 'g', 'a', 'b', "'", ',', '^', '=', '-', '_', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '>', '<', 'z', 'Z', '`', ':', '|',
    '(', ')', '{', '}', '[', ']', '.', '~', 'H', 'L', 'M', 'O', 'P', 'S', 'T', 'u', 'v', '!', '"', ' ', ';',
}

ABC notation 규칙 상 의미를 가지고 있다고 판단되는 문자들을 규정   
문장 간 구분을 위한 문자로 ';' 지정

### 규칙에 위배되는 문자 목록

In [9]:
tune_body = df['body'].apply(lambda x: ';'.join(x)).tolist()
print(sorted(set(';'.join(tune_body)).difference(char_vocab)))

['#', '*', '+', '@', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'W', 'X', '\\', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'w', 'x', 'y']


그 자체로 의미를 확인할 수 없는 문자들이 존재함을 확인   
먼저, 노이즈 데이터에 해당하는 annotation을 검사

### Annotation 제거

In [10]:
annot = set()
for tune_body in df['body']:
    for line in tune_body:
        annot.update(set(re.findall('!([^!]*)!', line)))
        annot.update(set(re.findall('"([^"]*)"', line)))
print(annot)

{'D/dim', 'Slower', '', '_Forte', 'G7', 'D7/b9', 'end ', 'A7/c+', 'Gm', 'Cm', 'F7', 'Cry of the hounds', ' D', 'f/+', 'E', 'Eb', '    Fine', 'Bm/a', '(a+)', 'fine', 'C7', 'P', '/@>.5A7', '(b)', 'C/g', 'Bm/d', '#~', 'S', 'Am/g+', 'Ab', 'DC al (S)', 'Sign', 'rall.', 'b', 'D/c+', 'C#7', 'Cm6', 'A7)', 'D.C', 'F#m', '<', 'segno', 'C', '~natural', 'F', '(Bm)', '/@<.8(D7)', 'Dm G7', '_F', '3', 'C/e', 'Adim', 'C  drone', 'd', 'g', '^[Ad lib...', 'C7/g', 'D#/dim', 'D.C. ', 'G#/dim', 'G7/b', 'D7/a', 'Daug', ' Em', 'Bb/f', 'G  C', 'Bm', 'Am/e', 'd#', 'f#', '_Pia.', 'Dm', 'C#', 'after ~', 'G7/d', '_For.', 'D/f+', 'g#', '(C)', 'E/dim', '^Minore', 'D', 'D/g', 'p', 'Em/d', 'Edim', '  Fine', 'F/c', 'G)', '^...]', 'D/nF', 'mf', 'A', 'a', '_Pia', '/@>.5A', 'e', ' ', '_x', 'Slow', 'A/c+', 'Dm/f', 'Bb7', 'D6', 'cresc.', '   S', 'e d+ d c+', 'G/(pl', 'Gm/bb', 'a tempo.', 'dim', 'B', '    S', 'c#', 'H', 'G/dim', 'C/c', 'Bm/f+', 'A7', 'Em7', 'eb', 'Em7/d', 'F#', 'D7/f+', '   Fine', 'Chorus.', '/@<.5D7', 'a#'

!...! 또는 "..." 조건을 만족하는 문자열을 추출했을 때,   
annotation으로 규정하기 어려운 긴 문자열을 다수 확인할 수 있음

In [11]:
violated_index = list()
df['body'] = df['body'].apply(lambda x: ';'.join(x))

for i, tune_body in zip(df.index, df['body']):
    annot = set()
    annot.update(set(re.findall('!([^!]*)!', tune_body)))
    annot.update(set(re.findall('"([^"]*)"', tune_body)))
    if max([len(s) for s in annot]+[0]) > 8:
        violated_index.append(i)
print('Songs with violated representations:', len(violated_index))

Songs with violated representations: 22


이후 전처리 시 편의성을 위해 ';' 구분자와 함께 문장 통합

규칙에 위배되었다고 판단되는 긴 문자열의 최소 기준을 9로 지정했을 때,   
해당 표현을 포함하는 곡의 수가 22개에 불과하기 때문에 제거해도 무방하다고 판단

In [12]:
df.drop(index=violated_index, axis=0, inplace=True)
df['body'] = df['body'].apply(lambda x: re.sub('!([^!]*)!','',x))
df['body'] = df['body'].apply(lambda x: re.sub('"([^"]*)"','',x))

print(df.shape)
df.head()

(5267, 5)


,T,M,L,K,body
0,The Enchanted Valley,2/4,1/16,Gm,G3-A (Bcd=e) | f4 (g2dB) | ({d}c3-B) G2-E2 | F...
1,Fare You Well,2/4,1/16,D,f-g | a3-b g3-a | f4 e3-d | d3-c A3-B | c4 d3-...
2,The Little Heathy Hill,C,1/8,Gm,B/2-c/2 | d2 d>-c B2 A-B | (GBAG) F2 D-F | (G>...
3,The Little Girl of my Heart,4/4,1/8,D,F-G | A-dd>-c d2 cd | e-fgg (f2 d>e) | f-d (c/...
4,The Fun at Donnybrook,6/8,1/8,Gm,B/2-A/2 | G>FD C>D^F | G3z2B/2-c/2 | B>cd cAG ...


규칙에 위배되는 22개 곡을 지우고 annotation을 전부 제거

## 문자 집합 생성

In [13]:
df['body'] = df['body'].apply(lambda x: str(x).replace('\\',''))
df['body'] = df['body'].apply(lambda x: str(x).replace('!',''))
df['body'] = df['body'].apply(lambda x: str(x).strip())

blackslash를 의미하는 '\\' 문자를 포함해 불필요하다고 판단되는 문자들을 추가로 제거

In [14]:
char_vocab = sorted(set(';'.join(df['body'].tolist())))
print(char_vocab)

[' ', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


처음 규정했던 문자 목록에 포함되지 않는 문자들이 여전히 존재하지만,   
'K:Dm'와 같이 전혀 불필요한 문자가 아니기도 하고, 수정 또는 제외 등의 처리가 어렵다고 판단되어 유지하기로 판단

In [15]:
df.to_csv('data/abc_char.csv', index=False)

전처리 완료 후 데이터 저장